The equations that we are trying solve are those of the paper titled "Diffuse-charge dynamics in electrochemical systems" by Bazant, Thornton and Ajdari published in https://journals.aps.org/pre/abstract/10.1103/PhysRevE.70.021506

In [1]:
from fipy import *
m = Grid1D(nx=100,Lx=2.0)
v = 1
epsilon= 0.05
delta = 0.1
x = m.cellCenters[0]

The set of equations is 

\begin{equation}
\begin{split}
\frac{\partial c}{\partial t}
&= \varepsilon\frac{\partial}{\partial x} \left(\frac{\partial c}{\partial x} + \rho\frac{\partial \phi}{\partial x}\right)\\
\frac{\partial \rho}{\partial t} 
&=  \varepsilon\frac{\partial}{\partial x} \left(\frac{\partial\rho}{\partial x}  + c \frac{\partial \phi}{\partial x}\right)\\
-\varepsilon^2 \frac{\partial^2\phi}{\partial x^2} &= \rho
\end{split}
\end{equation}

With the initial conditions

\begin{equation}
\begin{split}
c(x,0) &= 1\\
\rho(x,0) &= 0\\
\phi(x,0) &= v(x-1)\\
\end{split}
\end{equation}


and the boundary conditions

\begin{equation}
\begin{split}
\frac{\partial c}{\partial x} + \rho\frac{\partial \phi}{\partial x} &= 0\\
\frac{\partial \rho}{\partial x} + c\frac{\partial \phi}{\partial x} &= 0\\
\text{where both fluxes are zero at} &\quad\text{$x = 0$ and $x=2$}\\
\end{split}
\end{equation}

\begin{equation}
\phi = +v - \varepsilon\delta \frac{\partial \phi}{\partial x} \quad\text{at $x = 2$}
\end{equation}

\begin{equation}
\phi = -v + \varepsilon\delta \frac{\partial \phi}{\partial x} \quad\text{at $x = 0$}
\end{equation}

In [23]:
##Initial conditions
c = CellVariable(name="c", mesh=m , value=1.0, hasOld=True)

rho = CellVariable(name="rho", mesh=m , value=1., hasOld=True)

phi = CellVariable(name="phi",mesh=m, hasOld=True)
phi.setValue(v*(x-1))

#vi =Viewer((phi, c, rho),datamin=-1.1, datamax=1.1)

No-flux conditions are satisfied by default in FiPy. Constraint on $\phi$ is a [Robin condition](https://www.ctcms.nist.gov/fipy/documentation/USAGE.html#applying-robin-boundary-conditions).

In [19]:
##Boundary conditions
# c.faceGrad.constrain(-rho.faceValue*phi.faceGrad, where=m.facesLeft)
# c.faceGrad.constrain(-rho.faceValue*phi.faceGrad, where=m.facesRight)

# rho.faceGrad.constrain(-c.faceValue*phi.faceGrad, where=m.facesLeft)
# rho.faceGrad.constrain(-c.faceValue*phi.faceGrad, where=m.facesRight)

#phi.faceGrad.constrain((phi.faceValue+v)/(epsilon*delta), where=m.facesLeft)
#phi.faceGrad.constrain((phi.faceValue-v)/(-epsilon*delta), where=m.facesRight)

Calculate `cellDistanceVectors` manually because of issue [#706](https://github.com/usnistgov/fipy/issues/706) 

In [ ]:
from fipy.tools import numerix
MA = numerix.MA

tmp = MA.repeat(m._faceCenters[..., numerix.NewAxis,:], 2, 1)
cellToFaceDistanceVectors = tmp - numerix.take(m._cellCenters, m.faceCellIDs, axis=1)

tmp = numerix.take(m._cellCenters, m.faceCellIDs, axis=1)
tmp = tmp[..., 1,:] - tmp[..., 0,:]
cellDistanceVectors = MA.filled(MA.where(MA.getmaskarray(tmp), cellToFaceDistanceVectors[:, 0], tmp))

In [ ]:
mask = m.exteriorFaces
Gamma = FaceVariable(mesh=m, value=epsilon*epsilon)
Gamma.setValue(0., where=mask)
dPf = FaceVariable(mesh=m,
                   value=m._faceToCellDistanceRatio * cellDistanceVectors)
n = m.faceNormals
a = m.faceNormals
b = FaceVariable(mesh=m, value=epsilon * delta, rank=0)
g = FaceVariable(mesh=m, value=v, rank=0)
RobinCoeff = mask * epsilon * epsilon * n / (-dPf.dot(a) + b)

In [20]:
##Equation

Drho = epsilon*rho
Dc   = epsilon*c

eq1 = TransientTerm(var=c) == DiffusionTerm(coeff=epsilon, var=c) +DiffusionTerm(coeff=Drho, var=phi)
eq2 = TransientTerm(var=rho) == DiffusionTerm(coeff=epsilon, var=rho) +DiffusionTerm(coeff=Dc, var=phi)
eq3 = (DiffusionTerm(coeff=Gamma, var=phi) + (RobinCoeff * g).divergence
       - ImplicitSourceTerm(coeff=(RobinCoeff).divergence, var=phi) + ImplicitSourceTerm(coeff=1.0, var=rho)) == 0
eqns = eq1 & eq2 & eq3 

In [21]:
##Solve
#vi =Viewer((phi, c, rho),datamin=-1.1, datamax=1.1)
#from builtins import range

res = 1e+10
restol= 1e-13

for t in range(100):
    c.updateOld()
    rho.updateOld()
    phi.updateOld()
    while res > restol:
        res = eqns.sweep(dt=1e-15)
        print(res)
    
#if __name__=='__main__':
#        vi.plot()
#        raw_input("No-flux -- stationary. Press <return> to proceed..")


0.9977111011742081
5.210154151104807e+52
3.732246565196109e+94
2.732172022077301e+117
4.174789496284858e+121
8.336282317882696e+129
1.8262406026277956e+117
1.2467051786290536e+128
4.9592150756157276e+151
8.514249947026511e+140
6.151196898588066e+133
9.068402753909994e+151
1.0360423734597117e+143
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
nan


However the solution method above gave some errors, originated with the warning "self.factorizeFnc(*args, **kwargs)" 